imports

In [ ]:
# imports
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# show first 5 rows of lorry_df
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes (customer_locations) via nearest neighbour sorting

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

states and constraints

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# function: finding next nearest customer node
def nearest_customer(currentState, customerList):
    #initialise dist dict
    dist = {}
    
    # loop through customerList
    for i in customerList:
        # check for all where customer != currentState
        if i != currentState:
            # update dist with available customer index and their relative weights
            dist.update({i:euclidean[i,currentState]})

    # initialise temp list
    temp = []

    # loop through keys of dist 
    for i in dist.keys():
        # add weights to temp
        temp.append(dist[i])
    
    # get lowest weight
    _shortestDist = min(temp)
    
    # find relative index of lowest weight
    nearestCustomerIndex = np.where(euclidean[currentState]==_shortestDist)
    
    # return next index with relative weight
    return int(nearestCustomerIndex[0]), _shortestDist

In [ ]:
# test nearest_customer()
nearest_customer(1, [2, 3, 1, 5])

In [ ]:
# function: finding the nearest depot
def nearest_depot(currentState, depotList):
    # intialise dist dict
    dist = {}

    # loop through depotList
    for i in depotList:
        # update dist with depot and their relative distance values
        dist.update({i:euclidean[currentState, i]})
    
    # intialise temp list
    temp = []

    # loop through dist.keys()
    for i in dist.keys():
        # add values of each dist.keys() to temp 
        temp.append(dist[i])

    # get lowest depot weight
    _shortestDist = min(temp)

    # get the relative node index of the closest depot
    _nearestDepotIndex = np.where(euclidean[currentState]==_shortestDist)

    # return next depot index with relative weight
    return int(_nearestDepotIndex[0]), _shortestDist

In [ ]:
# test near_depot()
nearest_depot(1, depot_locations)

greedy search (best-first-search)

In [ ]:
# intialise routes dict
routes = {}

# loop through lorry_df.index 
for i in lorry_df.index:
    # update routes with key: lorry_id, value: capacity
    routes.update({lorry_df['lorry_id'][i]:lorry_df['capacity'][i]})

In [ ]:
# display journeys so far
routes

In [ ]:
print(edges[0][2])

In [ ]:
def routing(currState, toState, edges):
    visitedState = [currState]
    visitedEdge = []
    edgeMemory = []
    traversed = []
    routeWeights = []
    queue = [currState]

    while currState != toState:
        print(f'toState not found, pathfinding from node {currState}...')
        q = queue.pop(0)
        print(f'q: {q}')
        
        currEdges = {}
        for edge in list(np.where(links_df[['id1', 'id2']]==q)[0]):
            currEdges.update({edge:[edges[edge][0], edges[edge][1]]})
        print(f'currEdges dict: {currEdges}')
        edgeMemory.append(currEdges)

        for edge in currEdges:
            if edge not in visitedEdge:
                visitedEdge.append(edge)
                if currEdges[edge][0] not in visitedState and currEdges[edge][1] in visitedState:
                    print(f'currEdges[{edge}][0] selected...')
                    currState = currEdges[edge][0]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        print(f'edges[{edge}][0] solution found!')
                        currState = toState
                        break 
                if currEdges[edge][1] not in visitedState and edges[edge][0] in visitedState:
                    print(f'currEdges[{edge}][1] selected...')
                    currState = currEdges[edge][1]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        print(f'edges[{edge}][1] solution found!')
                        currState = toState
                        break
    
        print(f'visitedEdge list: {visitedEdge}')
        print(f'visitedState list: {visitedState}')
        print(f'\n') 

    print(edgeMemory)


routing(124, 10, edges)

In [ ]:
edgeMemory = [{308: [124, 208], 309: [124, 340], 310: [124, 542]}, {43: [14, 208], 308: [124, 208], 467: [204, 208]}, {309: [124, 340], 604: [286, 340], 677: [340, 542]}, {310: [124, 542], 677: [340, 542], 834: [473, 542]}, {36: [11, 14], 43: [14, 208], 44: [14, 36]}, {466: [204, 545], 467: [204, 208], 468: [204, 555]}, {603: [286, 565], 604: [286, 340], 605: [286, 315]}, {496: [222, 473], 834: [473, 542]}, {33: [11, 545], 34: [11, 411], 35: [11, 199], 36: [11, 14]}, {44: [14, 36], 96: [36, 480]}, {33: [11, 545], 466: [204, 545], 876: [545, 555]}, {153: [60, 555], 468: [204, 555], 754: [394, 555], 862: [516, 555], 876: [545, 555]}, {603: [286, 565], 701: [354, 565], 767: [409, 565]}, {563: [262, 315], 605: [286, 315], 646: [315, 552]}, {495: [222, 396], 496: [222, 473], 497: [222, 476]}, {34: [11, 411], 266: [106, 411]}, {35: [11, 199], 455: [199, 633]}, {96: [36, 480], 223: [89, 480], 833: [472, 480]}, {32: [10, 60], 151: [59, 60], 152: [60, 627], 153: [60, 555]}]
print(edgeMemory[0])

In [ ]:
temp = [0, 1, 2, 3, 4]

def route(lorryId, allocatedCustomers):
    currentId = lorryId
    operatingDepot = lorryId.split('-')[0]
    currentCapacity = float(lorry_df[lorry_df['lorry_id']==lorryId].capacity)
    currentCpm = float(lorry_df[lorry_df['lorry_id']==lorryId].cpm)
    currentState = int(operatingDepot)
    nextCustomerCapacity = 0
    while currentCapacity >= nextCustomerCapacity and len(allocatedCustomers)>0:
        nextCustomer = nearest_customer(currentState, allocatedCustomers)
        print(nextCustomer)
        # need for a routing function ... 
        print(allocatedCustomers)
        break


route("523-0", temp)